In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Overview

In this notebook, we want to provide an overview what HugeCTR framework is, its features and benefits. We will use HugeCTR to train a basic neural network architecture and deploy the saved model to Triton Inference Server. 

<b>Learning Objectives</b>:
* Adopt NVTabular workflow to provide input files to HugeCTR
* Define HugeCTR neural network architecture
* Train a deep learning model with HugeCTR
* Deploy HugeCTR to Triton Inference Server

### Why using HugeCTR?

HugeCTR is a GPU-accelerated recommender framework designed to distribute training across multiple GPUs and nodes and estimate Click-Through Rates (CTRs).<br>

HugeCTR offers multiple advantages to train deep learning recommender systems:
1. **Speed**: HugeCTR is a highly efficient framework written C++. We experienced up to 10x speed up. HugeCTR on a NVIDIA DGX A100 system proved to be the fastest commercially available solution for training the architecture Deep Learning Recommender Model (DLRM) developed by Facebook.
2. **Scale**: HugeCTR supports model parallel scaling. It distributes the large embedding tables over multiple GPUs or multiple nodes. 
3. **Easy-to-use**: Easy-to-use Python API similar to Keras. Examples for popular deep learning recommender systems architectures (Wide&Deep, DLRM, DCN, DeepFM) are available.

### Other Features of HugeCTR

HugeCTR is designed to scale deep learning models for recommender systems. It provides a list of other important features:
* Proficiency in oversubscribing models to train embedding tables with single nodes that don’t fit within the GPU or CPU memory (only required embeddings are prefetched from a parameter server per batch)
* Asynchronous and multithreaded data pipelines
* A highly optimized data loader.
* Supported data formats such as parquet and binary
* Integration with Triton Inference Server for deployment to production


### Getting Started

In this example, we will train a neural network with HugeCTR. We will use NVTabular for preprocessing.

#### Preprocessing and Feature Engineering with NVTabular

We use NVTabular to `Categorify` our categorical input columns.

In [5]:
# External dependencies
import os
import shutil
import gc

import nvtabular as nvt
import cudf
import numpy as np
import pandas as pd

from os import path
from sklearn.model_selection import train_test_split

from nvtabular.utils import download_file

We define our base directory, containing the data.

In [3]:
# path to store raw and preprocessed data
BASE_DIR = "/model/data/"

If the data is not available in the base directory, we will download and unzip the data.

In [3]:
download_file(
    "http://files.grouplens.org/datasets/movielens/ml-25m.zip", os.path.join(BASE_DIR, "ml-25m.zip")
)

downloading ml-25m.zip: 262MB [02:01, 2.15MB/s]                                                                                                                                                                 
unzipping files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:05<00:00,  1.48files/s]


## Preparing the dataset with NVTabular

First, we take a look at the movie metadata.

Let's load the movie ratings.

In [4]:
ratings = cudf.read_csv(os.path.join(BASE_DIR, "ml-25m", "ratings.csv"))
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [5]:
!ls /model/data/ml-25m

README.txt	   genome-tags.csv  movies.csv	 tags.csv
genome-scores.csv  links.csv	    ratings.csv


In [6]:
movies = pd.read_csv(os.path.join(BASE_DIR, "ml-25m", "movies.csv"))
movies = movies[['movieId', 'genres']]
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
movies['num_genre'] = movies['genres'].apply(lambda x: len(x))
movies = movies[movies['num_genre'] > 1]
movies['genres'] = movies['genres'].apply(lambda x: np.random.choice(np.array(x), 2, replace=False))

In [7]:
movies.drop(columns='num_genre', inplace=True)
movies = cudf.from_pandas(movies)
movies.head()

,movieId,genres
0,1,"[Fantasy, Comedy]"
1,2,"[Children, Adventure]"
2,3,"[Comedy, Romance]"
3,4,"[Romance, Drama]"
5,6,"[Thriller, Action]"


We drop the timestamp column and split the ratings into training and test dataset. We use a simple random split.

In [8]:
ratings = ratings.drop("timestamp", axis=1)
ratings.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [10]:
combined = movies.merge(ratings, how='right', on=['movieId'])
combined.dropna(how='any', inplace=True)
combined.head()

,userId,movieId,rating,genres
0,187,87522,3.5,"[Romance, Comedy]"
1,187,87529,3.5,"[Adventure, Comedy]"
2,187,87869,4.0,"[Comedy, Crime]"
3,187,87876,4.0,"[Comedy, Adventure]"
4,187,87911,4.0,"[Drama, Thriller]"


In [11]:
train, valid = train_test_split(combined, test_size=0.2, random_state=42)
train.head()

,userId,movieId,rating,genres
22828912,148460,7153,3.5,"[Fantasy, Drama]"
6187299,39954,3477,4.0,"[Comedy, Drama]"
4398075,28983,1127,3.0,"[Sci-Fi, Adventure]"
3879844,25431,1201,3.5,"[Adventure, Action]"
8768032,57127,2628,2.0,"[Action, Adventure]"


We save our train and valid datasets as parquet files on disk, and below we will read them in while initializing the Dataset objects.

In [12]:
train.to_parquet(BASE_DIR + "train.parquet")
valid.to_parquet(BASE_DIR + "valid.parquet")

In [13]:
del train
del valid
gc.collect()

46

Let's define our categorical and label columns. Note that in that example we do not have numerical columns.

In [14]:
CATEGORICAL_COLUMNS = ["userId", "movieId", "genres"]
LABEL_COLUMNS = ["rating"]

Let's add Categorify op for our categorical features, userId, movieId.

In [15]:
cat_features = CATEGORICAL_COLUMNS >> nvt.ops.Categorify()

The ratings are on a scale between 1-5. We want to predict a binary target with 1 are all ratings >=4 and 0 are all ratings <=3. We use the LambdaOp for it.

In [16]:
ratings = nvt.ColumnGroup(["rating"]) >> (lambda col: (col > 3).astype("int8"))

We can visualize our calculation graph.

In [17]:
output = cat_features + ratings
#(output).graph

We initialize our NVTabular workflow.

In [18]:
workflow = nvt.Workflow(output)

We initialize NVTabular Datasets, and use the part_size parameter, which defines the size read into GPU-memory at once, in nvt.Dataset.

In [19]:
train_dataset = nvt.Dataset(BASE_DIR + "train.parquet", part_size="100MB")
valid_dataset = nvt.Dataset(BASE_DIR + "valid.parquet", part_size="100MB")

First, we collect the training dataset statistics.

In [20]:
%%time

workflow.fit(train_dataset)

CPU times: user 2.32 s, sys: 384 ms, total: 2.71 s
Wall time: 10.5 s


This step is slightly different for HugeCTR. HugeCTR expect the categorical input columns as `int64` and continuous/label columns as `float32`  We can define output datatypes for our NVTabular workflow.

In [21]:
dict_dtypes = {}

for col in ["userId", "movieId"]:
    dict_dtypes[col] = np.int64

for col in LABEL_COLUMNS:
    dict_dtypes[col] = np.float32

Note: We do not have numerical output columns

In [22]:
train_dir = os.path.join(BASE_DIR, "train")
valid_dir = os.path.join(BASE_DIR, "valid")
if path.exists(train_dir):
    shutil.rmtree(train_dir)
if path.exists(valid_dir):
    shutil.rmtree(valid_dir)

In addition, we need to provide the data schema to the output calls. We need to define which output columns are `categorical`, `continuous` and which is the `label` columns. NVTabular will write metadata files, which HugeCTR requires to load the data and optimize training.

In [23]:
workflow.transform(train_dataset).to_parquet(
    output_path=BASE_DIR + "train/",
    shuffle=nvt.io.Shuffle.PER_PARTITION,
    cats=CATEGORICAL_COLUMNS,
    labels=LABEL_COLUMNS,
    dtypes=dict_dtypes,
)
workflow.transform(valid_dataset).to_parquet(
    output_path=BASE_DIR + "valid/",
    shuffle=False,
    cats=CATEGORICAL_COLUMNS,
    labels=LABEL_COLUMNS,
    dtypes=dict_dtypes,
)

In [35]:
!ls $BASE_DIR/train

0.94b5f166a02e42f2b04d94b668e60004.parquet  _metadata
_file_list.txt				    _metadata.json


In [36]:
dff = pd.read_parquet(BASE_DIR + "/train/0.94b5f166a02e42f2b04d94b668e60004.parquet")

In [37]:
dff.head()

,userId,movieId,genres,rating
0,89679,11175,"[5, 8]",1.0
1,86828,4385,"[5, 15]",1.0
2,66856,736,"[11, 16]",1.0
3,31213,669,"[13, 4]",1.0
4,99412,1033,"[15, 8]",1.0


## Scaling Accelerated training with HugeCTR

HugeCTR is a deep learning framework dedicated to recommendation systems. It is written in CUDA C++. As HugeCTR optimizes the training in CUDA++, we need to define the training pipeline and model architecture and execute it via the commandline. We will use the Python API, which is similar to Keras models.

HugeCTR has three main components:
* Solver: Specifies various details such as active GPU list, batchsize, and model_file
* Optimizer: Specifies the type of optimizer and its hyperparameters
* DataReader: Specifies the training/evaludation data
* Model: Specifies embeddings, and dense layers. Note that embeddings must precede the dense layers

**Solver**

Let's take a look on the parameter for the `Solver`. We should be familiar from other frameworks for the hyperparameter.

```
solver = hugectr.CreateSolver(
- vvgpu: GPU indices used in the training process, which has two levels. For example: [[0,1],[1,2]] indicates that two nodes are used in the first node. GPUs 0 and 1 are used while GPUs 1 and 2 are used for the second node. It is also possible to specify non-continuous GPU indices such as [0, 2, 4, 7]  
- batchsize: Minibatch size used in training
- max_eval_batches: Maximum number of batches used in evaluation. It is recommended that the number is equal to or bigger than the actual number of bathces in the evaluation dataset.
If max_iter is used, the evaluation happens for max_eval_batches by repeating the evaluation dataset infinitely.
On the other hand, with num_epochs, HugeCTR stops the evaluation if all the evaluation data is consumed    
- batchsize_eval: Maximum number of batches used in evaluation. It is recommended that the number is equal to or
  bigger than the actual number of bathces in the evaluation dataset
- mixed_precision: Enables mixed precision training with the scaler specified here. Only 128,256, 512, and 1024 scalers are supported
)
```

**Optimizer**

The optimizer is the algorithm to update the model parameters. HugeCTR supports the common algorithms.


```
optimizer = CreateOptimizer(
- optimizer_type: Optimizer algorithm - Adam, MomentumSGD, Nesterov, and SGD 
- learning_rate: Learning Rate for optimizer
)
```

**DataReader**

The data reader defines the training and evaluation dataset.


```
reader = hugectr.DataReaderParams(
- data_reader_type: Data format to read
- source: The training dataset file list. IMPORTANT: This should be a list
- eval_source: The evaluation dataset file list.
- check_type: The data error detection machanism (Sum: Checksum, None: no detection).
- slot_size_array: The list of categorical feature cardinalities
)
```

**Model**

We initialize the model with the solver, optimizer and data reader:

```
model = hugectr.Model(solver, reader, optimizer)
```

We can add multiple layers to the model with `model.add` function. We will focus on:
- `Input` defines the input data
- `SparseEmbedding` defines the embedding layer
- `DenseLayer` defines dense layers, such as fully connected, ReLU, BatchNorm, etc.

**HugeCTR organizes the layers by names. For each layer, we define the input and output names.**

Input layer:

This layer is required to define the input data.

```
hugectr.Input(
    label_dim: Number of label columns
    label_name: Name of label columns in network architecture
    dense_dim: Number of continous columns
    dense_name: Name of contiunous columns in network architecture
    data_reader_sparse_param_array: Configuration how to read sparse data and its names
)
```

SparseEmbedding:

This layer defines embedding table

```
hugectr.SparseEmbedding(
    embedding_type: Different embedding options to distribute embedding tables 
    workspace_size_per_gpu_in_mb: Maximum embedding table size in MB
    embedding_vec_size: Embedding vector size
    combiner: Intra-slot reduction op
    sparse_embedding_name: Layer name
    bottom_name: Input layer names
    optimizer: Optimizer to use
)
```

DenseLayer:

This layer is copied to each GPU and is normally used for the MLP tower.

```
hugectr.DenseLayer(
    layer_type: Layer type, such as FullyConnected, Reshape, Concat, Loss, BatchNorm, etc.
    bottom_names: Input layer names
    top_names: Layer name
    ...: Depending on the layer type additional parameter can be defined
)
```

This is only a short introduction in the API. You can read more in the official docs: [Python Interface](https://github.com/NVIDIA/HugeCTR/blob/master/docs/python_interface.md) and [Layer Book](https://github.com/NVIDIA/HugeCTR/blob/master/docs/hugectr_layer_book.md)

## Let's define our model

We walked through the documentation, but it is useful to understand the API. Finally, we can define our model. We will write the model to `./model.py` and execute it afterwards.

We need the cardinalities of each categorical feature to assign as `slot_size_array` in the model below.

In [24]:
from nvtabular.ops import get_embedding_sizes

embeddings = get_embedding_sizes(workflow)
print(embeddings)

({'movieId': (29434, 509), 'userId': (162542, 512)}, {'genres': (20, 16)})


Let's clear the directory and create the output folders

We use `graph_to_json` to convert the model to a JSON configuration, required for the inference.

In [42]:
%%writefile './model.py'

import hugectr
from mpi4py import MPI  # noqa

solver = hugectr.CreateSolver(
    vvgpu=[[3]],
    batchsize=2048,
    batchsize_eval=2048,
    max_eval_batches=160,
    i64_input_key=True,
    use_mixed_precision=False,
    repeat_dataset=True,
)
optimizer = hugectr.CreateOptimizer(optimizer_type=hugectr.Optimizer_t.Adam)
reader = hugectr.DataReaderParams(
    data_reader_type=hugectr.DataReaderType_t.Parquet,
    source=["/model/data/train/_file_list.txt"],
    eval_source="/model/data/valid/_file_list.txt",
    check_type=hugectr.Check_t.Non,
    slot_size_array=[162542, 29434, 20],
)


model = hugectr.Model(solver, reader, optimizer)

model.add(
    hugectr.Input(
        label_dim=1,
        label_name="label",
        dense_dim=0,
        dense_name="dense",
        data_reader_sparse_param_array=[
            hugectr.DataReaderSparseParam("data1", nnz_per_slot=[1, 1, 2], is_fixed_length=False, slot_num=3)
        ],
    )
)
model.add(
    hugectr.SparseEmbedding(
        embedding_type=hugectr.Embedding_t.LocalizedSlotSparseEmbeddingHash,
        workspace_size_per_gpu_in_mb=200,
        embedding_vec_size=16,
        combiner="sum",
        sparse_embedding_name="sparse_embedding1",
        bottom_name="data1",
        optimizer=optimizer,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.Reshape,
        bottom_names=["sparse_embedding1"],
        top_names=["reshape1"],
        leading_dim=48,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.InnerProduct,
        bottom_names=["reshape1"],
        top_names=["fc1"],
        num_output=128,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.ReLU,
        bottom_names=["fc1"],
        top_names=["relu1"],
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.InnerProduct,
        bottom_names=["relu1"],
        top_names=["fc2"],
        num_output=128,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.ReLU,
        bottom_names=["fc2"],
        top_names=["relu2"],
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.InnerProduct,
        bottom_names=["relu2"],
        top_names=["fc3"],
        num_output=1,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.BinaryCrossEntropyLoss,
        bottom_names=["fc3", "label"],
        top_names=["loss"],
    )
)
model.compile()
model.summary()
model.fit(max_iter=2000, display=100, eval_interval=200, snapshot=1900)
model.graph_to_json(graph_config_file="/model/movielens_hugectr/1/movielens.json")

Overwriting ./model.py


We trained our model.

In [4]:
!python model.py

====================================================Model Init=====================================================
[06d02h00m45s][HUGECTR][INFO]: Global seed is 3979425881
[06d02h00m45s][HUGECTR][INFO]: Device to NUMA mapping:
  GPU 3 ->  node 1

[06d02h00m57s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
[06d02h00m57s][HUGECTR][INFO]: Start all2all warmup
[06d02h00m57s][HUGECTR][INFO]: End all2all warmup
[06d02h00m57s][HUGECTR][INFO]: Using All-reduce algorithm NCCL
Device 3: Tesla T4
[06d02h00m57s][HUGECTR][INFO]: num of DataReader workers: 1
[06d02h00m57s][HUGECTR][INFO]: Vocabulary size: 191996
[06d02h00m57s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=3276800
[06d02h00m57s][HUGECTR][INFO]: All2All Warmup Start
[06d02h00m57s][HUGECTR][INFO]: All2All Warmup End
===================================================Model Compile===================================================
[06d02h00m59s][HUGECTR][INFO]: gpu0 start to init embedding
[06d02h00m59s][HUGECTR][INFO]: 

After training terminates, we can see that multiple `.model` files and folders are generated. We need to move them inside `1` folder under the `movielens_hugectr` folder. Let's create these folders first.

Now we move our saved `.model` files inside `1` folder.

In [44]:
!mv *.model /model/movielens_hugectr/1/